# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
file_path = ("../WeatherPy/output_data/cities.csv")
weather_df = pd.read_csv(file_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key = g_key)

In [61]:
type(weather_df["Humidity"])

pandas.core.series.Series

In [92]:
figure_layout = {'width': '990px',
                'height': '600px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '1 auto 0 auto'
                }


fig = gmaps.figure(layout=figure_layout, center=(20,0), zoom_level= 2)

coordinates = weather_df[["Lat" , "Lng"]].sort_values(by = ["Lng"])

weight = weather_df["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weight, dissipating= False, max_intensity = 110 , point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='1 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [93]:
weather_df.describe()

,City_ID,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
count,547.000000,547.000000,5.470000e+02,547.000000,547.000000,547.000000,547.000000,547.000000
mean,299.391225,61.561243,1.639896e+09,72.837294,19.128865,18.804632,10.954004,3.772011
std,170.979232,40.783011,3.378514e+02,21.572712,33.525336,92.035178,17.403748,2.766479
min,0.000000,0.000000,1.639895e+09,6.000000,-54.800000,-175.200000,-43.120000,0.000000
25%,152.500000,15.000000,1.639896e+09,63.000000,-8.133350,-63.069050,1.070000,1.720000
50%,304.000000,86.000000,1.639896e+09,78.000000,22.266700,22.957300,16.940000,3.110000
75%,449.500000,100.000000,1.639896e+09,89.000000,47.683300,102.272600,24.115000,5.175000
max,588.000000,100.000000,1.639897e+09,100.000000,78.218600,179.316700,44.030000,16.270000


In [106]:
# narrow down the cities list based on these criteria:
#     1. Wind speed less then 10 mph
#     2. Temperature is between 21 and 27 celsius
#     3. No cloud
shortlist_df = weather_df.loc[(weather_df["Wind Speed"] < 10)&
                              (weather_df["Max Temp"] > 21)&
                             (weather_df["Max Temp"] < 27)&
                             (weather_df["Cloudiness"] == 0)
                             ]
print(f"Narrow down to {len(shortlist_df)} from {len(weather_df)} cities")
shortlist_df.reset_index()

Narrow down to 12 from 547 cities


,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,60,67,tezu,0,IN,1639895739,22,27.9167,96.1667,24.93,1.64
1,77,86,cape town,0,ZA,1639895483,68,-33.9258,18.4232,23.40,1.03
2,101,112,port elizabeth,0,ZA,1639895832,77,-33.9180,25.5701,21.72,3.60
3,124,140,doha,0,QA,1639895798,61,25.2867,51.5333,24.99,7.72
4,171,190,east london,0,ZA,1639895949,60,-33.0153,27.9116,22.52,5.14
5,255,281,geraldton,0,AU,1639896136,50,-28.7667,114.6000,26.23,9.77
6,398,437,tura,0,IN,1639896429,42,25.5198,90.2201,22.88,2.11
7,402,442,rayagada,0,IN,1639896433,39,19.1667,83.4167,26.42,1.52
8,423,464,savantvadi,0,IN,1639896509,25,15.9000,73.8167,26.81,2.85
9,452,493,saraipali,0,IN,1639896531,31,21.3333,83.0000,23.24,3.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [128]:
params = {"keyword": "hotel",
          "types": "hotel", 
          "radius": 5000, 
          "key": g_key}

shortlist_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in shortlist_df.iterrows():
    coordinates = (row["Lat"], row["Lng"])
    params["location"] = coordinates,
    response = requests.get(base_url, params)

    print(f"Searching for hotel within 5 kms from {shortlist_df.loc[index, 'City'].title()}'s coordinate:", end = " ")
    try:
        print(coordinates)
        print("Successful!")
        pprint(response.json())
    except:
        print("Failed!")
        pass
    
    
# lat = seattle_geo["results"][0]["geometry"]["location"]["lat"]

<ipython-input-128-3afda66623d0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortlist_df["Hotel Name"] = ""


Searching for hotel within 5 kms from Tezu's coordinate: (27.9167, 96.1667)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel within 5 kms from Cape Town's coordinate: (-33.9258, 18.4232)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel within 5 kms from Port Elizabeth's coordinate: (-33.918, 25.5701)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel within 5 kms from Doha's coordinate: (25.2867, 51.5333)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel within 5 kms from East London's coordinate: (-33.0153, 27.9116)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel within 5 kms from Geraldton's coordinate: (-28.7667, 114.6)
Successful!
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Searching for hotel with

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
